# Group 16 Milestone 1

In [1]:
import os
import os.path
import zipfile
import requests
import json
import pandas as pd
from urllib.request import urlretrieve
import glob
import re

In [2]:
# Necessary metadata
url = "https://api.figshare.com/v2/articles/14096681"
headers = {"Content-Type": "application/json"}
output_directory = "../data/"
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]

In [3]:
%%time
files_to_dl = "data.zip"
if not os.path.isfile(output_directory + files_to_dl):
    for file in files:
        if file["name"] == files_to_dl:
            os.makedirs(output_directory, exist_ok=True)
            urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 34 µs, sys: 62 µs, total: 96 µs
Wall time: 98.2 µs


In [4]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, files_to_dl), "r") as f:
    f.extractall(output_directory)
os.remove("../data/observed_daily_rainfall_SYD.csv")

CPU times: user 14.7 s, sys: 996 ms, total: 15.7 s
Wall time: 16.4 s


In [5]:
%%time
files = glob.glob("../data/*.csv")
columns = ["time", "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)"]
df = pd.concat((pd.read_csv(file, index_col=0, usecols=columns)
                .assign(model=re.findall(r"[^\/]+(?=\_daily)", file)[0])
                for file in files)
              )

CPU times: user 39.3 s, sys: 5.68 s, total: 45 s
Wall time: 48.6 s


In [6]:
df

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
...,...,...,...,...,...,...
2014-12-27 12:00:00,-30.157068,-29.214660,153.1250,154.3750,6.689683e+00,SAM0-UNICON
2014-12-28 12:00:00,-30.157068,-29.214660,153.1250,154.3750,7.862555e+00,SAM0-UNICON
2014-12-29 12:00:00,-30.157068,-29.214660,153.1250,154.3750,1.000503e+01,SAM0-UNICON


## Runtimes of CSV compilation


| Team Member | OS           | RAM | Processor        | Is SSD | Wall Time Taken |
|-------------|--------------|-----|------------------|--------|-----------------|
| Nikita      | Ubuntu 20.04 | 8GB | 8th Gen Core i7  | Yes    | 1min 32s        |
| Margot      |              |     |                  |        |                 |
| Thea        |              |     |                  |        |                 |
| Kiran       |    MacOS Big Sur          |   8GB  |     Apple M1 chip            |    Yes    |        1min 40s         |

## EDA for R

In [7]:
%load_ext rpy2.ipython

**In order to select which method is the most appropriate to transfer the dataframe from python to R, we chose to try all methods and observe which method had a better performance:**

### Parquet Method

In [8]:
%%time
df.to_parquet("../data/rainfall.parquet")

CPU times: user 23.4 s, sys: 5.58 s, total: 29 s
Wall time: 31.7 s


In [9]:
%%R


UsageError: %%R is a cell magic, but the cell body is empty. Did you mean the line magic %R (single %)?


### Feather Method

In [10]:
%%time
import pyarrow.feather as feather
feather.write_feather(df, '../data/rainfall.feather')

CPU times: user 4.29 s, sys: 4.13 s, total: 8.42 s
Wall time: 5.78 s


### Arrow Exchange Method

In [19]:
import rpy2.robjects.conversion
import pyarrow
import rpy2.rinterface
import rpy2_arrow.pyarrow_rarrow as pyra
from rpy2.robjects.packages import importr

In [25]:
%%time
r_frame = pyra.converter.py2rpy(pyarrow.Table.from_pandas(df))

CPU times: user 4.81 s, sys: 3.77 s, total: 8.59 s
Wall time: 9.97 s


In [26]:
%%R -i r_frame
head(r_frame)

Table
6 rows x 7 columns
$lat_min <double>
$lat_max <double>
$lon_min <double>
$lon_max <double>
$rain (mm/day) <double>
$model <string>
$time <string>

See $metadata for additional Schema metadata


In [19]:
%%R
library(arrow)
library(dplyr)
rdf <- open_dataset("../data/rainfall.parquet") |> filter(lat_min > -30) |> collect()